# Lakeflow Connect: Demo del Conector de SharePoint

Este notebook demuestra cómo usar el **Conector de SharePoint de Databricks** (actualmente en Beta) para ingerir varios tipos de archivos desde Microsoft SharePoint en tablas Delta con gobernanza de Unity Catalog. Haz una copia de este notebook para ejecutarlo tú mismo.

Consulta nuestra documentación pública [aquí](https://docs.databricks.com/aws/en/ingestion/sharepoint). 

## Lo que Aprenderás

* **Ingerir PDFs no estructurados** - Leer archivos PDF y analizarlos usando funciones de IA
* **Sincronizar archivos Excel** - Cargar archivos Excel individuales en tablas Delta
* **Ingerir archivos CSV** - Fusionar múltiples archivos estructurados con el mismo esquema
* **Ingesta Incremental Automática usando Lakeflow SDP** - Orquestar toda tu ingesta automáticamente usando Lakeflow Spark Declarative Pipelines

## Requisitos Previos

* **Conexión de Unity Catalog**: Esta demo usa la conexión `brickfood_sharepoint_connection` de Unity Catalog
* **Databricks Runtime**: 17.3 o superior (requerido para el Conector de SharePoint)

## Acerca del Conector de SharePoint

El Conector de SharePoint soporta:
* Ingesta por lotes y streaming (Auto Loader, spark.read, COPY INTO)
* Archivos estructurados (CSV, Excel), semi-estructurados (JSON) y no estructurados (PDF, imágenes)
* Gobernanza y seguridad de Unity Catalog
* Inferencia y evolución automática de esquemas

## Acceso a archivos de demostración 
Lo siguiente es SOLO PARA EMPLEADOS INTERNOS DE DATABRICKS:
* **Acceso a SharePoint**: Para ver los mismos archivos de demostración usados en este notebook de Demo, consulta [aquí](https://drive.google.com/drive/u/0/folders/1Pa_zHBiUDTFeuP5OSoJM1OgoEoup0usV).  
* **Acceso a SharePoint**: El `brickfood_sharepoint_connection` te permitirá ingerir nuestros archivos de demostración. Si deseas previsualizar los archivos dentro de un entorno de SharePoint, visita [aquí](https://docs.google.com/document/d/1H7Kfy0YJBQta3rdDpVjkNcPVRn10x8tytTB6DJ6WPmE/edit?tab=t.0) o contacta a **@Jason Ping**, **@Matt Zhang**, o **@Mason Force** para obtener acceso a nuestro entorno interno de demostración de SharePoint. 

---

## 1. Ingerir PDFs No Estructurados y Analizar con IA

Esta sección demuestra cómo:
1. Leer archivos PDF desde SharePoint como archivos binarios
2. Guardarlos en una tabla Delta
3. Usar funciones de IA SQL para analizar y extraer contenido estructurado de los PDFs

In [0]:
# Read all PDF files from SharePoint as binary files
pdf_df = (spark.read
    .format("binaryFile")
    .option("databricks.connection", "brickfood_sharepoint_connection")
    .option("recursiveFileLookup", True)
    .option("pathGlobFilter", "*.pdf")  # Only ingest PDF files
    .load("https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared%20Documents/Forms/AllItems.aspx")
)

# Save the PDF files to a Delta table for persistent storage
pdf_df.write \
    .mode("overwrite") \
    .saveAsTable("aircraft_maintence_logs_pdfs")

print("✅ PDF files saved to Delta table: aircraft_maintence_logs_pdfs")

# Display the DataFrame to see the PDF files
display(pdf_df)

✅ PDF files saved to Delta table: aircraft_maintence_logs_pdfs


path,modificationTime,length,content
https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N148CK_28.pdf,2025-12-03T23:17:22Z,183180,JVBERi0xLjQKJSBjcmVhdGVkIGJ5IFBpbGxvdyAxMC4zLjAgUERGIGRyaXZlcgo0IDAgb2JqPDwKL1R5cGUgL0NhdGFsb2cKL1BhZ2VzIDUgMCBSCj4+ZW5kb2JqCjUgMCBvYmo8PAovVHlwZSAvUGFnZXMKL0NvdW50IDEKL0s= (truncated)
https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N216BY_64.pdf,2025-12-03T23:17:48Z,176506,JVBERi0xLjQKJSBjcmVhdGVkIGJ5IFBpbGxvdyAxMC4zLjAgUERGIGRyaXZlcgo0IDAgb2JqPDwKL1R5cGUgL0NhdGFsb2cKL1BhZ2VzIDUgMCBSCj4+ZW5kb2JqCjUgMCBvYmo8PAovVHlwZSAvUGFnZXMKL0NvdW50IDEKL0s= (truncated)
https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N189IT_100.pdf,2025-12-03T23:17:20Z,175052,JVBERi0xLjQKJSBjcmVhdGVkIGJ5IFBpbGxvdyAxMC4zLjAgUERGIGRyaXZlcgo0IDAgb2JqPDwKL1R5cGUgL0NhdGFsb2cKL1BhZ2VzIDUgMCBSCj4+ZW5kb2JqCjUgMCBvYmo8PAovVHlwZSAvUGFnZXMKL0NvdW50IDEKL0s= (truncated)
https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N116IK_72.pdf,2025-12-03T23:17:21Z,174974,JVBERi0xLjQKJSBjcmVhdGVkIGJ5IFBpbGxvdyAxMC4zLjAgUERGIGRyaXZlcgo0IDAgb2JqPDwKL1R5cGUgL0NhdGFsb2cKL1BhZ2VzIDUgMCBSCj4+ZW5kb2JqCjUgMCBvYmo8PAovVHlwZSAvUGFnZXMKL0NvdW50IDEKL0s= (truncated)
https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N102SV_81 (1).pdf,2025-12-03T23:17:34Z,172762,JVBERi0xLjQKJSBjcmVhdGVkIGJ5IFBpbGxvdyAxMC4zLjAgUERGIGRyaXZlcgo0IDAgb2JqPDwKL1R5cGUgL0NhdGFsb2cKL1BhZ2VzIDUgMCBSCj4+ZW5kb2JqCjUgMCBvYmo8PAovVHlwZSAvUGFnZXMKL0NvdW50IDEKL0s= (truncated)
https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N105JY_76.pdf,2025-12-03T23:17:34Z,164837,JVBERi0xLjQKJSBjcmVhdGVkIGJ5IFBpbGxvdyAxMC4zLjAgUERGIGRyaXZlcgo0IDAgb2JqPDwKL1R5cGUgL0NhdGFsb2cKL1BhZ2VzIDUgMCBSCj4+ZW5kb2JqCjUgMCBvYmo8PAovVHlwZSAvUGFnZXMKL0NvdW50IDEKL0s= (truncated)
https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N144CD_16.pdf,2025-12-03T23:17:38Z,160259,JVBERi0xLjQKJSBjcmVhdGVkIGJ5IFBpbGxvdyAxMC4zLjAgUERGIGRyaXZlcgo0IDAgb2JqPDwKL1R5cGUgL0NhdGFsb2cKL1BhZ2VzIDUgMCBSCj4+ZW5kb2JqCjUgMCBvYmo8PAovVHlwZSAvUGFnZXMKL0NvdW50IDEKL0s= (truncated)
https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N187XO_19.pdf,2025-12-03T23:18:00Z,157838,JVBERi0xLjQKJSBjcmVhdGVkIGJ5IFBpbGxvdyAxMC4zLjAgUERGIGRyaXZlcgo0IDAgb2JqPDwKL1R5cGUgL0NhdGFsb2cKL1BhZ2VzIDUgMCBSCj4+ZW5kb2JqCjUgMCBvYmo8PAovVHlwZSAvUGFnZXMKL0NvdW50IDEKL0s= (truncated)
https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N166IM_2.pdf,2025-12-03T23:17:19Z,154115,JVBERi0xLjQKJSBjcmVhdGVkIGJ5IFBpbGxvdyAxMC4zLjAgUERGIGRyaXZlcgo0IDAgb2JqPDwKL1R5cGUgL0NhdGFsb2cKL1BhZ2VzIDUgMCBSCj4+ZW5kb2JqCjUgMCBvYmo8PAovVHlwZSAvUGFnZXMKL0NvdW50IDEKL0s= (truncated)
https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N214FM_55.pdf,2025-12-03T23:17:24Z,153504,JVBERi0xLjQKJSBjcmVhdGVkIGJ5IFBpbGxvdyAxMC4zLjAgUERGIGRyaXZlcgo0IDAgb2JqPDwKL1R5cGUgL0NhdGFsb2cKL1BhZ2VzIDUgMCBSCj4+ZW5kb2JqCjUgMCBvYmo8PAovVHlwZSAvUGFnZXMKL0NvdW50IDEKL0s= (truncated)


### Analizar PDFs usando `ai_parse_document()`

Al ingerir archivos no estructurados desde SharePoint (como PDFs, documentos de Word o archivos de PowerPoint) usando el conector estándar de SharePoint con formato binaryFile, el contenido de los archivos se almacena como datos binarios sin procesar.

Para preparar estos archivos para cargas de trabajo de IA—como RAG, búsqueda, clasificación o comprensión de documentos—puedes analizar fácilmente el contenido binario en una salida estructurada y consultable simplemente aplicando `ai_parse_document()` en la columna `content`.

In [0]:
%sql
-- Parse PDF content using AI and extract structured information. 
SELECT
  *,
  ai_parse_document(content) AS parsed_content
FROM
  aircraft_maintence_logs_pdfs
LIMIT 5 -- limiting to only the first 5 PDFs

path modificationTime length parsed_content https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N102SV_81 (1).pdf 2025-12-03T23:17:34Z 172762 {"document":{"elements":[{"bbox":[{"coord":[914,80,1126,106],"page_id":0}],"content":"AIRCRAFT MAINTENANCE LOGBOOK","description":null,"id":0,"type":"page_header"},{"bbox":[{"coord":[89,212,1985,2527],"page_id":0}],"content":" Date Tech/Hour Time Total Time Description of Work Performed 03/15/2025 32.2.4 32.9.3 Bench tested empennage attach bolts. Found in nominal condition. Tested brake master cylinder. Found in secure condition. After abnormal engine reading, found alternator field circuit slightly chafing. Removed and replaced component EAW service manual. Test passed. I certify that this aircraft has been inspected in accordance with the scope and detail of Part 43 Appendix D and have determined it to be in a condition for safe flight. David Phillips 9265909XJP N-Number: N102SY ","description":null,"id":1,"type":"table"}],"pages":[{"id":0,"image_uri":null}]},"error_status":null,"metadata":{"file_metadata":null,"id":"cf82d65a-3ac7-4a02-9bdc-d7db2d8df68e","version":"2.0"}} https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N105JY_76.pdf 2025-12-03T23:17:34Z 164837 {"document":{"elements":[{"bbox":[{"coord":[921,86,1129,106],"page_id":0}],"content":"AIRCRAFT MAINTENANCE LOGBOOK","description":null,"id":0,"type":"text"},{"bbox":[{"coord":[66,215,1990,2529],"page_id":0}],"content":" Date Tech. Area Time Time Description of Work Performed 03/27/2025 56.9 68.4 Performed Annual/Condition Inspection IAW Part 43, Appendix D. Bench tested exhaust manifold. Found in airworthy condition. Bench tested fuselage skin. Found in within specified tolerance condition. I certify that this aircraft has been inspected in accordance with the scope and detail of Part 43 appendix D and have determined it to be in a condition for safe flight. Monique Smith 7016580AJ-P N-Number: N105JY ","description":null,"id":1,"type":"table"}],"pages":[{"id":0,"image_uri":null}]},"error_status":null,"metadata":{"file_metadata":null,"id":"4fcf8da7-6b24-401c-8279-dd99c222176d","version":"2.0"}} https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N107KY_9.pdf 2025-12-03T23:17:48Z 136645 {"document":{"elements":[{"bbox":[{"coord":[923,86,1126,103],"page_id":0}],"content":"AIRCRAFT MAINTENANCE LOGBOOK","description":null,"id":0,"type":"text"},{"bbox":[{"coord":[64,198,1990,2529],"page_id":0}],"content":" N-Number: N107K Tech-Mech Time Title Description of work performed Date: 03/27/2025 19:03 203.5 Tested fuselage skin. Found in satisfactory condition. I certify that this aircraft has been inspected in accordance with the seope and detail of Part 43 appendix D and have determined it to be in a condition for safe flight. Joseph Wright 4644867XAJP ","description":null,"id":1,"type":"table"}],"pages":[{"id":0,"image_uri":null}]},"error_status":null,"metadata":{"file_metadata":null,"id":"f91a71fb-2870-416f-849a-7de4006a4e74","version":"2.0"}} https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared Documents/logbook_N116IK_72.pdf 2025-12-03T23:17:21Z 174974 {"document":{"elements":[{"bbox":[{"coord":[928,86,1137,107],"page_id":0}],"content":"AIRCRAFT MAINTENANCE LOGBOOK","description":null,"id":0,"type":"page_header"},{"bbox":[{"coord":[101,210,509,266],"page_id":0}],"content":"N-Number: N116IEK","description":null,"id":1,"type":"text"},{"bbox":[{"coord":[555,266,646,293],"page_id":0}],"content":"Tach/Hour Time","description":null,"id":2,"type":"text"},{"bbox":[{"coord":[972,266,1029,293],"page_id":0}],"content":"Total Time","description":null,"id":3,"type":"text"},{"bbox":[{"coord":[1451,266,1625,293],"page_id":0}],"content":"Description of Work Performed","description":null,"id":4,"type":"text"},{"bbox":[{"coord":[101,370,401,450],"page_id":0}],"content":"01/31/2025","description":null,"id":5,"type":"text"},{"bb

También puedes usar `ai_parse_document()` **dentro de Lakeflow Spark Declarative Pipelines para habilitar el análisis incremental**. A medida que llegan nuevos archivos desde SharePoint, se analizan automáticamente cuando tu pipeline se actualiza.

---

## 2. Sincronizar Archivos Excel Individuales a Tablas Delta

Esta sección demuestra cómo:
1. Leer un archivo Excel específico desde SharePoint usando Python `spark.read()` o SQL `read_files()`

Los archivos Excel soportan opciones como `headerRows` y `dataAddress` para especificar qué hoja y rango leer.

In [0]:
# Read a specific Excel file from SharePoint
excel_df = (spark.read
    .format("excel")
    .option("databricks.connection", "brickfood_sharepoint_connection")
    .option("headerRows", 1)  # First row contains headers
    .option("inferSchema", True)  # Automatically infer column types
    .load("https://databricks977.sharepoint.com/sites/brickfood-demo-site/Sample%20Files/Example%20Excel%20Files/sales_data_sample.xlsx")
)

# Save the Excel data to a Delta table
excel_df.write \
    .mode("overwrite") \
    .saveAsTable("brickfood_excel_report")

print("✅ Excel file saved to Delta table: brickfood_excel_report")

# Display the Excel data
display(excel_df)

✅ Excel file saved to Delta table: brickfood_excel_report


ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
10107,30,95.700000000000000,2,2871.0000000000000,2/24/2003 0:00,Shipped,1,2,2003,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,null,NYC,NY,10022,USA,NA,Yu,Kwai,Small
10121,34,81.349999999999994,5,2765.9000000000000,5/7/2003 0:00,Shipped,2,5,2003,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,null,Reims,null,51100,France,EMEA,Henriot,Paul,Small
10134,41,94.740000000000000,2,3884.3400000000000,7/1/2003 0:00,Shipped,3,7,2003,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,null,Paris,null,75508,France,EMEA,Da Cunha,Daniel,Medium
10145,45,83.260000000000000,6,3746.7000000000000,8/25/2003 0:00,Shipped,3,8,2003,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,null,Pasadena,CA,90003,USA,NA,Young,Julie,Medium
10159,49,100.000000000000000,14,5205.2700000000004,10/10/2003 0:00,Shipped,4,10,2003,Motorcycles,95,S10_1678,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,null,San Francisco,CA,null,USA,NA,Brown,Julie,Medium
10168,36,96.660000000000000,1,3479.7600000000000,10/28/2003 0:00,Shipped,4,10,2003,Motorcycles,95,S10_1678,Technics Stores Inc.,6505556809,9408 Furth Circle,null,Burlingame,CA,94217,USA,NA,Hirano,Juri,Medium
10180,29,86.130000000000000,9,2497.7700000000000,11/11/2003 0:00,Shipped,4,11,2003,Motorcycles,95,S10_1678,Daedalus Designs Imports,20.16.1555,"184, chausse de Tournai",null,Lille,null,59000,France,EMEA,Rance,Martine,Small
10188,48,100.000000000000000,1,5512.3200000000000,11/18/2003 0:00,Shipped,4,11,2003,Motorcycles,95,S10_1678,Herkku Gifts,+47 2267 3215,"Drammen 121, PR 744 Sentrum",null,Bergen,null,N 5804,Norway,EMEA,Oeztan,Veysel,Medium
10201,22,98.570000000000000,2,2168.5400000000000,12/1/2003 0:00,Shipped,4,12,2003,Motorcycles,95,S10_1678,Mini Wheels Co.,6505555787,5557 North Pendale Street,null,San Francisco,CA,null,USA,NA,Murphy,Julie,Small
10211,41,100.000000000000000,14,4708.4399999999996,1/15/2004 0:00,Shipped,1,1,2004,Motorcycles,95,S10_1678,Auto Canal Petit,(1) 47.55.6555,"25, rue Lauriston",null,Paris,null,75016,France,EMEA,Perrier,Dominique,Medium


In [0]:
%sql
-- Create a table from an Excel file using SQL
CREATE OR REPLACE TABLE brickfood_excel_report_sql AS
SELECT * FROM read_files(
  "https://databricks977.sharepoint.com/sites/brickfood-demo-site/Sample%20Files/Example%20Excel%20Files/sales_data_sample.xlsx",
  `databricks.connection` => "brickfood_sharepoint_connection",
  format => "excel",
  headerRows => 1,
  inferSchema => true,
  schemaEvolutionMode => "none"
);


-- Query the newly created table
SELECT * FROM brickfood_excel_report_sql LIMIT 10

ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
10107,30,95.700000000000000,2,2871.0000000000000,2/24/2003 0:00,Shipped,1,2,2003,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,null,NYC,NY,10022,USA,NA,Yu,Kwai,Small
10121,34,81.349999999999994,5,2765.9000000000000,5/7/2003 0:00,Shipped,2,5,2003,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,null,Reims,null,51100,France,EMEA,Henriot,Paul,Small
10134,41,94.740000000000000,2,3884.3400000000000,7/1/2003 0:00,Shipped,3,7,2003,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,null,Paris,null,75508,France,EMEA,Da Cunha,Daniel,Medium
10145,45,83.260000000000000,6,3746.7000000000000,8/25/2003 0:00,Shipped,3,8,2003,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,null,Pasadena,CA,90003,USA,NA,Young,Julie,Medium
10159,49,100.000000000000000,14,5205.2700000000004,10/10/2003 0:00,Shipped,4,10,2003,Motorcycles,95,S10_1678,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,null,San Francisco,CA,null,USA,NA,Brown,Julie,Medium
10168,36,96.660000000000000,1,3479.7600000000000,10/28/2003 0:00,Shipped,4,10,2003,Motorcycles,95,S10_1678,Technics Stores Inc.,6505556809,9408 Furth Circle,null,Burlingame,CA,94217,USA,NA,Hirano,Juri,Medium
10180,29,86.130000000000000,9,2497.7700000000000,11/11/2003 0:00,Shipped,4,11,2003,Motorcycles,95,S10_1678,Daedalus Designs Imports,20.16.1555,"184, chausse de Tournai",null,Lille,null,59000,France,EMEA,Rance,Martine,Small
10188,48,100.000000000000000,1,5512.3200000000000,11/18/2003 0:00,Shipped,4,11,2003,Motorcycles,95,S10_1678,Herkku Gifts,+47 2267 3215,"Drammen 121, PR 744 Sentrum",null,Bergen,null,N 5804,Norway,EMEA,Oeztan,Veysel,Medium
10201,22,98.570000000000000,2,2168.5400000000000,12/1/2003 0:00,Shipped,4,12,2003,Motorcycles,95,S10_1678,Mini Wheels Co.,6505555787,5557 North Pendale Street,null,San Francisco,CA,null,USA,NA,Murphy,Julie,Small
10211,41,100.000000000000000,14,4708.4399999999996,1/15/2004 0:00,Shipped,1,1,2004,Motorcycles,95,S10_1678,Auto Canal Petit,(1) 47.55.6555,"25, rue Lauriston",null,Paris,null,75016,France,EMEA,Perrier,Dominique,Medium


---

## 3. Ingesta por Lotes de Múltiples Archivos CSV con el Mismo Esquema

Esta sección demuestra cómo:
1. Leer múltiples archivos CSV desde una carpeta de SharePoint
2. Fusionarlos en una tabla Delta existente
3. Usar COPY INTO para carga incremental idempotente

Esto es útil cuando tienes múltiples archivos con la misma estructura que necesitan ser consolidados en una sola tabla.

In [0]:
# Read all CSV files from a SharePoint folder
csv_df = (spark.read
    .format("csv")
    .option("databricks.connection", "brickfood_sharepoint_connection")
    .option("pathGlobFilter", "*.csv")  # Only read CSV files
    .option("recursiveFileLookup", True)  # Search subdirectories
    .option("inferSchema", True)  # Automatically infer column types
    .option("header", True)  # First row contains headers
    .load("https://databricks977.sharepoint.com/sites/brickfood-demo-site/Sample%20Files/Sample%20CSV%20with%20Same%20Schema")
)

# Create or replace the Delta table with CSV data
csv_df.write \
    .mode("overwrite") \
    .saveAsTable("brickfood_csv_data")

print("✅ CSV files saved to Delta table: brickfood_csv_data")

# Display the combined CSV data
display(csv_df)

✅ CSV files saved to Delta table: brickfood_csv_data


TransactionID,Date,Product,Category,Quantity,UnitPrice,Total
TRX-1004,2023-04-06T06:17:00.000Z,HDMI Cable,Accessories,2,214,428
TRX-1007,2023-04-17T21:58:15.000Z,Mouse,Accessories,14,1279,17906
TRX-1022,2023-04-14T15:16:50.000Z,HDMI Cable,Accessories,16,155,2480
TRX-1030,2023-04-12T20:49:03.000Z,Laptop,Accessories,4,160,640
TRX-1035,2023-04-17T20:01:28.000Z,Monitor,Electronics,11,1298,14278
TRX-1044,2023-04-13T01:10:36.000Z,Monitor,Electronics,3,25,75
TRX-1047,2023-04-09T21:58:55.000Z,Keyboard,Accessories,6,111,666
TRX-1049,2023-04-17T07:10:33.000Z,Mouse,Electronics,2,984,1968
TRX-1006,2023-02-21T21:20:31.000Z,HDMI Cable,Accessories,3,1428,4284
TRX-1013,2023-02-05T15:57:15.000Z,HDMI Cable,Electronics,4,1158,4632


### Ingesta Incremental con COPY INTO

`COPY INTO` proporciona carga incremental idempotente - rastrea automáticamente qué archivos han sido procesados y solo carga archivos nuevos. Esto es perfecto para ingesta continua de datos donde se agregan nuevos archivos CSV a SharePoint con el tiempo.

In [0]:
%sql
-- Create the table if it doesn't exist
CREATE TABLE IF NOT EXISTS sharepoint_sample_csv_incremental;

-- Incrementally ingest new CSV files (only processes files not yet loaded)
COPY INTO sharepoint_sample_csv_incremental
  FROM "https://databricks977.sharepoint.com/sites/brickfood-demo-site/Sample%20Files/Sample%20CSV%20with%20Same%20Schema"
  FILEFORMAT = CSV
  PATTERN = '*.csv'
  FORMAT_OPTIONS('header' = 'true', 'inferSchema' = 'true', 'databricks.connection' = "brickfood_sharepoint_connection")
  COPY_OPTIONS ('mergeSchema' = 'true');

-- Query the incrementally loaded data
SELECT * FROM sharepoint_sample_csv_incremental LIMIT 10

TransactionID,Date,Product,Category,Quantity,UnitPrice,Total
TRX-1020,2023-01-15T11:35:31.000Z,Monitor,Accessories,1,1254,1254
TRX-1036,2023-01-06T10:17:28.000Z,Mouse,Electronics,16,1259,20144
TRX-1006,2023-02-21T21:20:31.000Z,HDMI Cable,Accessories,3,1428,4284
TRX-1013,2023-02-05T15:57:15.000Z,HDMI Cable,Electronics,4,1158,4632
TRX-1041,2023-02-02T19:24:59.000Z,Mouse,Accessories,7,278,1946
TRX-1045,2023-02-03T03:48:04.000Z,Laptop,Electronics,18,505,9090
TRX-1010,2023-03-27T19:35:46.000Z,Monitor,Accessories,18,413,7434
TRX-1032,2023-03-02T16:37:57.000Z,Mouse,Electronics,8,88,704
TRX-1038,2023-03-18T02:55:23.000Z,Laptop,Electronics,8,264,2112
TRX-1004,2023-04-06T06:17:00.000Z,HDMI Cable,Accessories,2,214,428


---

## 4. Ingesta Automática e Incremental usando Lakeflow Spark Declarative Pipelines

Mientras que los ejemplos anteriores mostraron ingesta por lotes, **Lakeflow Spark Declarative Pipelines** te permite orquestar **ingesta automática e incremental** desde SharePoint. A medida que nuevos archivos llegan a un directorio de SharePoint (o cuando los archivos se actualizan), el pipeline los detecta e ingiere automáticamente.

### Beneficios Clave

* **Detección automática de archivos** - Los archivos nuevos o actualizados se descubren y procesan automáticamente
* **Procesamiento incremental** - Solo se procesan datos nuevos, no todo el conjunto de datos
* **Evolución de esquema** - Se adapta automáticamente a cambios de esquema en tus archivos
* **Orquestación** - Programación y gestión de dependencias integradas
* **Calidad de datos** - Expectativas y monitoreo integrados en el pipeline

### Casos de Uso

* **Tablas de streaming** - Para ingesta continua de PDFs, CSVs u otros archivos a medida que llegan
* **Vistas materializadas** - Para archivos Excel específicos que se actualizan periódicamente (ej., informes mensuales)

Los ejemplos a continuación muestran cómo definir tablas de pipeline usando sintaxis de Python y SQL.

### Lakeflow Spark Declarative Pipelines

Define tablas de streaming y vistas materializadas usando sintaxis SQL. Usa `CREATE OR REFRESH STREAMING TABLE` para ingesta continua y `CREATE OR REFRESH MATERIALIZED VIEW` para datos actualizados periódicamente.

NOTA: Los ejemplos de código a continuación deben ser copiados a un pipeline SDP para ser desplegados. Consulta nuestra Documentación de Lakeflow Spark Declarative Pipelines para aprender más sobre cómo desplegar un pipeline SDP.


In [0]:
%sql
-- NOTE: You must copy the code into an SDP pipeline to deploy. View our documentation to learn more.

-- Incrementally ingest new PDF files as they arrive
CREATE OR REFRESH STREAMING TABLE aircraft_maintenance_pdfs_streaming_sql
AS SELECT * FROM STREAM read_files(
  "https://databricks977.sharepoint.com/sites/brickfood-demo-site/Shared%20Documents/Forms/AllItems.aspx",
  format => "binaryFile",
  `databricks.connection` => "brickfood_sharepoint_connection",
  pathGlobFilter => "*.pdf"
);

In [0]:
%sql
-- NOTE: You must copy the code into an SDP pipeline to deploy. View our documentation to learn more.

-- Incrementally ingest CSV files with automatic schema inference and evolution
CREATE OR REFRESH STREAMING TABLE sharepoint_sample_csv_streaming_sql
AS SELECT * FROM STREAM read_files(
  "https://databricks977.sharepoint.com/sites/brickfood-demo-site/Sample%20Files/Sample%20CSV%20with%20Same%20Schema",
  format => "csv",
  `databricks.connection` => "brickfood_sharepoint_connection",
  pathGlobFilter => "*.csv",
  header => "true"
);

In [0]:
%sql
-- NOTE: You must copy the code into an SDP pipeline to deploy. View our documentation to learn more.

-- Read a specific Excel file from SharePoint in a materialized view
-- This will automatically refresh when the Excel file is updated
CREATE OR REFRESH MATERIALIZED VIEW sharepoint_sample_excel_materialized_sql
AS SELECT * FROM read_files(
  "https://databricks977.sharepoint.com/sites/brickfood-demo-site/Sample%20Files/Example%20Excel%20Files/sales_data_sample.xlsx",
  `databricks.connection` => "brickfood_sharepoint_connection",
  format => "excel",
  headerRows => 1,
  `cloudFiles.schemaEvolutionMode` => "none"
);

---

## Próximos Pasos y Recursos

### Lo que Has Aprendido

Has explorado exitosamente:
* ✅ Ingesta por lotes de archivos PDF, Excel y CSV desde SharePoint
* ✅ Análisis de documentos impulsado por IA con `ai_parse_document()`
* ✅ Carga incremental con COPY INTO
* ✅ Ingesta automática y orquestada con Lakeflow Spark Declarative Pipelines

### Funciones de IA Adicionales para Explorar

Una vez que hayas analizado tus documentos, explora estas funciones de IA para análisis adicional:
* `ai_summarize()` - Generar resúmenes de contenido de texto
* `ai_extract()` - Extraer entidades específicas de documentos
* `ai_classify()` - Clasificar documentos en categorías
* `ai_analyze_sentiment()` - Analizar el sentimiento en texto
* `ai_query()` - Consultar modelos de fundación para análisis personalizado

### Recursos

* **Documentación**: [Guía del Conector de SharePoint](https://docs.databricks.com/aws/en/ingestion/sharepoint)
* **Lakeflow Pipelines**: [Documentación de Spark Declarative Pipelines](https://www.databricks.com/product/data-engineering/spark-declarative-pipelines)
* **Funciones de IA**: [Referencia de Funciones de IA SQL](https://docs.databricks.com/aws/en/large-language-models/ai-functions)

### ¿Preguntas?

Para empleados internos de Databricks: Proporciona retroalimentación y preguntas en [#lakeflow-connect](https://databricks.enterprise.slack.com/archives/C05HQQEAZ0D)